# CarND Novermber Cohort Project 1: Lane Detection
## Functions used (including helper)
* Weight Image 
* Grayscale 
* Canny Edge Detection
* Region Selection
* Hough Lines
* Gaussion Smoothing
* Color Selection
    
    

In [47]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import os
images = os.listdir("test_images/")
print('images: ', images)

images:  ['solidWhiteCurve.jpg', 'solidWhiteRight.jpg', 'solidYellowCurve.jpg', 'solidYellowCurve2.jpg', 'solidYellowLeft.jpg', 'whiteCarLaneSwitch.jpg']


In [49]:
#reading an image
def readImage(image):
    image = mpimg.imread('test_images/' + image)
    gray = grayscale(image)
    blur_gray = apply_gaussian_blur(gray)
    edges = apply_canny(blur_gray)
    masked_image = apply_mask(edges, image, gray)
    hough_image, lines = apply_hough(masked_image)
    lines_edges_weighted = apply_lines_edges_weighted(image, edges, hough_image)
    plot_mask(edges, image, masked_image)
    plot_hough(hough_image, lines)
    plot_mask_and_hough(masked_image, hough_image)
    plot_weighted_image(lines_edges_weighted)
    return lines_edges_weighted

#Read each image and output the result
def readAllImages(images):
    for img in images:
        result = readImage(img)
        mpimg.imsave('results_rendered/' + '-ChangedParameters--'+img, result)

Now that we have made a loop to read all images we want to convert each image to a grayscale by taking off the RGB valus. By getting rid of the 3rd value we take its 3rd Dimension of color.

Then we apply a Guassion Blur filder to reduce noise from the image. It will also smooth out the edges which will help us with our canny edge detector later.
* The Gaussian Blur will take a kernel size of 5 to cover the larger area of the images. Kernel sizes are always odd numbers and generally range from 3,5 and 7
* We also want to give it a low and high threshold. The recommended ratio is either 1:2 or 1:3 low to high thresholds

In [50]:
#Apply Gaussian Blur
def apply_gaussian_blur(gray, kernel_size = 5):
    return gaussian_blur(gray, kernel_size)

Now that we've applied our Gaussian Blur we will apply canny using the image and the low and high thresholds.
* Any values below the low threshold will be discard and anything aboe the high threshold will be considereed edges
* We've set the low threshold to be 75 and the high threshold to be 175

In [51]:
#Define parameters for Canny Edge Detector
def apply_canny(blur_gray, low_threshold = 75, high_threshold = 175):
    return canny(blur_gray, low_threshold, high_threshold)

Now we will apply and image mask so it only considers the lane lines and plot the mask so we get our region of interest

In [52]:
def apply_mask(edges, image, gray):

    mask = np.zeros_like(edges)
    
    ignore_mask_color = 255 

    imshape = image.shape

    ysize = image.shape[0]
    xsize = image.shape[1]
    
    x_middle = xsize/2
    x_offset = 55
    
    y_middle = ysize/2
    y_offset = 45
    vertices = np.array([[(0, ysize),(x_middle - x_offset, y_middle + y_offset), (x_middle + x_offset, y_middle + y_offset), 
                          (xsize, ysize)]], dtype=np.int32)
    

    #fillPoly the mask
    (masked_image, mask) = region_of_interest(edges, vertices)
    return masked_image

def plot_mask(edges, image, masked_image):
    #Plotting each of the images using subplot
    fig = plt.figure()

    add = fig.add_subplot(3,1,1)
    edgeplot = plt.imshow(edges, cmap="gray")
    add.set_title('Edges')

    add = fig.add_subplot(3,1,2)
    add.set_title('Image')
    edgeplot = plt.imshow(image)

    add = fig.add_subplot(3,1,3)
    add.set_title('Masked Image')
    edgeplot = plt.imshow(masked_image, cmap="gray")
    plt.show()

Now that we've applied the image mask it's time to apply the hough transformation and create the lines on the masked image

In [53]:
def apply_hough(masked_image):
    theta = np.pi/180     
    rho = 1.8
    threshold = 75
    min_line_len = 20
    max_line_gap = 120

    #Create a line image with hough lines on it
    hough_image, lines = hough_lines(masked_image, rho, theta, threshold, min_line_len, max_line_gap)

    return hough_image, lines
    #Gave line_image color when we copied the shape of mask_image
    #In draw_lines we input color = [255, 0, 0] so that it will be red

def plot_hough(hough_image, lines):
    fig = plt.figure()

    b = fig.add_subplot(2,1,1)
    edgeplot = plt.imshow(lines)
    b.set_title('Lines')

    b = fig.add_subplot(2,2,2)
    b.set_title('Hough Image')
    edgeplot = plt.imshow(hough_image)
    plt.show()

We now have a line image and a masked image. Now we just apply the line image on top of the mask images and form the two images into one

In [54]:
def plot_mask_and_hough(masked_image, hough_image):
    fig = plt.figure()
    c = fig.add_subplot(1,2,1)
    edgeplot = plt.imshow(masked_image, cmap="gray")
    c.set_title('masked_image')

    c = fig.add_subplot(1,2,2)
    c.set_title('Hough_image')
    edgeplot = plt.imshow(hough_image)
    plt.show()

In [55]:
#Img is the output of hough lines to line images
#initial_image is the color_edges
def apply_lines_edges_weighted(image, edges, hough_image):
    lines_edges_weighted = weighted_img(hough_image, image)
    return lines_edges_weighted

def plot_lines_edges_weighted_hough_and_original():
    #Plot using subplots
    fig = plt.figure()
    d = fig.add_subplot(3,1,1)
    line_edgeplot = plt.imshow(hough_image)
    d.set_title('hough_image')

    d = fig.add_subplot(3,1,2)
    line_edgeplot = plt.imshow(color_edges)
    d.set_title('color_edges')

    d = fig.add_subplot(3,1,3)
    line_edgeplot = plt.imshow(lines_edges_weighted)
    d.set_title('lines_edges_weighted')

    plt.show()
    
def plot_weighted_image(lines_edges_weighted):
    plt.imshow(lines_edges_weighted)
    
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [56]:
#Converts image to grayscale
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#Applies canny transform
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

#Applies gaussian noise kernel, typically use kernel size of 3 or 5. (odd numbers only)
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)

    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:  #if it is 3 or 4 channels; RGB or RGBT
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count  #so you can fill multiple colors
    else:
        ignore_mask_color = 255

    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    #returning the image only where mask pixels are nonzero
    # Because we just filled the mask matrix with vertices in the *fillPoly* function.
    masked_image = cv2.bitwise_and(img, mask)
    return (masked_image, mask)


def draw_lines(img, lines, color=[255, 0, 0], thickness=11):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    x1_values = lines[:, :, 0]
    y1_values = lines[:, :, 1]
    x2_values = lines[:, :, 2]
    y2_values = lines[:, :, 3]


    global prev_left_x_min
    global prev_left_x_max
    global prev_right_x_min
    global prev_right_x_max

    y_max = max(np.amax(y1_values), np.amax(y2_values))
    y_min = max(np.amin(y1_values), np.amin(y2_values))
    left_slopes = []
    right_slopes = []

    right_lane = []
    left_lane = []

    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1)

            #if slope < 0:  #left lane has negative slope
            if (-0.9 < slope < -0.6):
                left_slopes.append(slope)
                left_lane.append(line)
            elif 0.45 < slope < 0.7: # right lane positive slope
                right_slopes.append(slope)
                right_lane.append(line)

    left_slopes_arr = np.array(left_slopes)
    right_slopes_arr = np.array(right_slopes)

    left_lines_arr = np.array(left_lane)
    right_lines_arr = np.array(right_lane)

    # Find average slopes
    left_lane_slope_average = np.mean(left_slopes_arr) 
    right_lane_slope_average = np.mean(right_slopes_arr)

    y_max = int(y_max)
    y_min = int(y_min)

    if len(left_lines_arr) > 0:
        #Compute Left lane intercept:
        y_intercept_left_lane_average = np.mean(left_lines_arr[:, :, 1] - (left_lines_arr[:, :, 0] * left_lane_slope_average))

        #Compute Left lane Xmax and Xmin:
        x_max_left = int((y_max - y_intercept_left_lane_average) / left_lane_slope_average)
        x_min_left = int((y_min - y_intercept_left_lane_average) / left_lane_slope_average)
            

        cv2.line(img, (x_max_left, y_max), (x_min_left, y_min), color, thickness)
        #Saves the last coordinate to keep the line going from new start point
        
        prev_left_x_max = x_max_left
        prev_left_x_min = x_min_left

    else:
        
        if prev_left_x_max is not None:
            print('prev_left_x_max is not None')
            cv2.line(img, (prev_left_x_max, y_max),(prev_left_x_min, y_min), color, thickness)
        else:
            print('nothing')
            
    if len(right_lines_arr) > 0:
        # Compute Right Lane Intercept:
        y_intercept_right_lane_average = np.mean(right_lines_arr[:, :, 1] - (right_lines_arr[:, :, 0] * right_lane_slope_average))

        # Compute Right lane Xmax and Xmin:
        x_max_right = int((y_max - y_intercept_right_lane_average) / right_lane_slope_average)
        x_min_right = int((y_min - y_intercept_right_lane_average) / right_lane_slope_average)

        cv2.line(img, (x_max_right, y_max), (x_min_right, y_min), color, thickness)
        prev_right_x_max = x_max_right
        prev_right_x_min = x_min_right
    else:
        if prev_right_x_max is not None:
            print('prev_right_x_max is not NONE')
            cv2.line(img, (prev_right_x_max, y_max), (prev_right_x_min, y_min), color, thickness)
        else:
            
            print('nothing right side empty')




def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        img shape will be 2D output from canny transform
    Returns an image with hough lines drawn.
    """
    hough_lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len,
                                  maxLineGap=max_line_gap)

    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)  

    draw_lines(line_img, hough_lines)
    return (line_img, hough_lines)

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [57]:

#Save previous coordinates as global variables in case program doesn't detect any lines fromp previous iteration
prev_left_x_min = None
prev_left_x_max = None
prev_right_x_min = None
prev_right_x_max = None

In [58]:
def process_image(image):
    gray = grayscale(image)
    blur_gray = apply_gaussian_blur(gray)
    edges = apply_canny(blur_gray)
    masked_image = apply_mask(edges, image, gray)
    hough_image, lines = apply_hough(masked_image)
    lines_edges_weighted = apply_lines_edges_weighted(image, edges, hough_image)
    return lines_edges_weighted

In [59]:

# Test on solidWhite
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

white_output = 'submit-solidWhiteRightx2.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video submit-solidWhiteRightx2.mp4
[MoviePy] Writing video submit-solidWhiteRightx2.mp4


 62%|█████████████████████████████████████████████████▎                              | 137/222 [00:01<00:01, 69.77it/s]

prev_left_x_max is not None


 93%|██████████████████████████████████████████████████████████████████████████▌     | 207/222 [00:02<00:00, 71.73it/s]

prev_left_x_max is not None


100%|███████████████████████████████████████████████████████████████████████████████▋| 221/222 [00:03<00:00, 73.26it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: submit-solidWhiteRightx2.mp4 

Wall time: 3.48 s


In [60]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [61]:

# Test on solid yellow left
yellow_output = 'submit-solidYellowLeftTestx2.mp4'
clip2 = VideoFileClip("solidYellowLeft.mp4")
yellow_clip = clip2.fl_image(process_image) #NOTE: this function expects color images!!
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video submit-solidYellowLeftTestx2.mp4
[MoviePy] Writing video submit-solidYellowLeftTestx2.mp4


100%|███████████████████████████████████████████████████████████████████████████████▉| 681/682 [00:09<00:00, 71.71it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: submit-solidYellowLeftTestx2.mp4 

Wall time: 9.87 s


In [62]:
HTML("""
<video width="1000" height="600" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [63]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


  0%|                                                                                          | 0/251 [00:00<?, ?it/s]

prev_right_x_max is not NONE


 10%|████████▍                                                                        | 26/251 [00:00<00:04, 46.56it/s]

prev_right_x_max is not NONE


...

prev_right_x_max is not NONE


 63%|██████████████████████████████████████████████████▎                             | 158/251 [00:04<00:03, 29.67it/s]

prev_right_x_max is not NONE
prev_right_x_max is not NONE


100%|████████████████████████████████████████████████████████████████████████████████| 251/251 [00:07<00:00, 35.01it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

Wall time: 8.01 s


In [64]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

# Reflection: 
I spent most of my time figuring out which functions to use from the list of helper functions then finding a way to put it all together. I'm fairly new to python and I've never used jupyter notebook before but thanfully with the help of my mentor, some information from the forums, the P1 slack group conversation and some ideas from JonathanCMitchell's Git repository I was able to get the environment installed and running fine fairly quickly and get started on the project. I had the idea of how to create the project for a while but it took me a lot of time to figure out how to implement it all and put it together on notebook. I'm pretty sattisfied with the result of the video tests overall. However, if I were to try to improve on one thing it would be the result of the challenge video. I tried just slightly changing the slope of the original formula for the base videos but it didn't work out as well as I had hoped for the challenge. I would definitely love to try and figure out a way to create curved lines that would automatically follow the traffic lines using hough and canny edge. I think it would be a further change in slope but I'm not entirely sure. All in all it was a great project and I'm very excited to continue learning and start the next project.